In [163]:
from torchmetrics.functional.text.rouge import rouge_score

In [2]:
preds = "My name is John"
target = "Is your name John"
rouge_score(preds, target, rouge_keys=('rouge1','rouge2'))

{'rouge1_fmeasure': tensor(0.7500),
 'rouge1_precision': tensor(0.7500),
 'rouge1_recall': tensor(0.7500),
 'rouge2_fmeasure': tensor(0.),
 'rouge2_precision': tensor(0.),
 'rouge2_recall': tensor(0.)}

In [35]:
from bpe_dataset_fft import BPEDataset
from bpe_tokenizer import BPETokenizer
from mixed_transformer_lm import TransformerLM
import torch
import numpy as np

In [4]:
max_vocab = 10000
max_len1 = 1024
max_len2 = 128

In [6]:
tokenizer = BPETokenizer(None, 
                         'BPE_vocab_10000.pickle',
                         max_vocab,
                         drop=0.0)

In [7]:
bpedataset = BPEDataset('dataset/dailymail/onebigfile_train.npy',
                        'dataset/dailymail/onebigfile_validation.npy',
                        max_len1,
                        max_len2,
                        batch_size = 32,
                        shuffle = True)
train, validation, train_loader, validation_loader = bpedataset.reset()


In [8]:
len(validation)

12597

In [9]:
model = TransformerLM(vocab_size = max_vocab,
                      max_len1 = max_len1,
                      max_len2 = max_len2,
                      dmodel = 256,
                      dk = 64,
                      dhead = 4,
                      dff = 512,
                      p = 0.1,
                      nlayer = 3,
                      kernel_type = 'vanilla')

In [13]:
tmp = torch.load('runs/run_mixed_lm/ckpt_dir/_2022-12-27_01-02-56/model_49.pickle',
                 map_location=torch.device('cpu'))

In [15]:
model.load_state_dict(tmp['model'])

<All keys matched successfully>

In [16]:
model.eval()

TransformerLM(
  (encoder_layers): ModuleList(
    (0): Encoder(
      (dropout): Dropout(p=0.1, inplace=False)
      (attention): F_Attention()
      (ff): FeedForward(
        (fc1): Linear(in_features=256, out_features=512, bias=True)
        (fc2): Linear(in_features=512, out_features=256, bias=True)
        (activation): ReLU()
      )
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (1): Encoder(
      (dropout): Dropout(p=0.1, inplace=False)
      (attention): F_Attention()
      (ff): FeedForward(
        (fc1): Linear(in_features=256, out_features=512, bias=True)
        (fc2): Linear(in_features=512, out_features=256, bias=True)
        (activation): ReLU()
      )
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (2): Encoder(
      (dropout): Dropout(p=0.1, inplace=False)
      (a

In [21]:
from tqdm import tqdm

In [26]:
with torch.no_grad():

    output_all = []
    z_all = []
    w_all = []

    for x, y, w in tqdm(validation_loader):
        z = y[:, 1:]
        output = model(x, y[:,:-1])                
        output = torch.argmax(output, dim=-1)

        for i in range(output.shape[0]):
            preds = tokenizer.detokenize(output[i][w[i]])
            target = tokenizer.detokenize(z[i][w[i]])
            print(preds, target)

  0%|▎                                                                                                                                                 | 1/393 [00:01<08:54,  1.36s/it]

prince royal posed who ,posed bombed new selfie taken new zealand rugby sevens coach sir gordon tietjens ,sport manawatu chief executive trevor shailer and massey university professor emeritus gary hermansson .the trio were watching the 800m women 's swimming style in glasgow on <eos>this ,england women ckey team on their beat england games games .<eos>ed hands smilfor the ,who amed the hockeyroos .smile glasgow aftermath of the win win .<eos> the royal ,29 ,photobombed a selfie between new zealand rugby sevens coach sir gordon tietjens ,sport manawatu chief executive trevor shailer and massey university professor emeritus gary hermansson .the men were watching the 800m women 's freestyle in glasgow .earlier harry met australian hockey team after they beat the english team .smiled and posed with players - nicknamed the hockeyroos - in the wake of their 3-0 victory .<eos>
viktoras bruzas ,38 ,is with murder of patrick ketgillian kettyle ,bruwere 38 surrey surrey ,were in the  wounds in 

  0%|▎                                                                                                                                                 | 1/393 [00:02<17:10,  2.63s/it]


KeyboardInterrupt: 

In [68]:
x, y, w = next(iter(validation_loader))

In [72]:
model(x[0:1,:], y[0:1,:-1])

tensor([[[-12.5802,  -0.2580,   4.2409,  ...,  -7.9273,  -9.6861,  -2.2647],
         [-14.5581,  -0.2826,   4.7183,  ..., -10.6025,  -8.1970,  -2.4363],
         [-13.4424,  -0.7465,   2.5710,  ...,  -6.8200,  -6.8260,   1.4469],
         ...,
         [-14.1142,  -0.5491,   2.9959,  ...,  -9.4087,  -9.3971,  -0.9885],
         [-12.5914,   0.6637,  12.7072,  ...,  -8.6436,  -9.0045,  -2.6813],
         [-12.9826,   1.1116,   6.4896,  ..., -10.1624,  -7.5825,  -1.0715]]],
       grad_fn=<MulBackward0>)

In [27]:
rouge_score(preds, target, rouge_keys=('rouge1','rouge2'))

{'rouge1_fmeasure': tensor(0.5806),
 'rouge1_precision': tensor(0.5625),
 'rouge1_recall': tensor(0.6000),
 'rouge2_fmeasure': tensor(0.2333),
 'rouge2_precision': tensor(0.2258),
 'rouge2_recall': tensor(0.2414)}

In [137]:
import heapq
class BeamSearch():
    def __init__(self, topk, x_len, max_len, model, tokenizer, max_queue=2000):
        self.topk = topk
        self.x_len = x_len
        self.max_len = max_len
        self.max_queue = max_queue
        self.model = model.eval().cpu()
        self.bos_idx = tokenizer.bos_idx
        self.eos_idx = tokenizer.eos_idx
        self.pad_idx = tokenizer.pad_idx
    
    def _nexttoken(self, x, y):
        '''
          x, y are 2D tensor 1*L including the token idx
        '''
        tmp = self.model(x, y)[0,-1,:]
        tmp = torch.topk(tmp, self.topk, dim=-1)   # use log(probability)
        
        pp = tmp.values.detach().numpy()
        pp = np.log(pp)
        idx = tmp.indices.detach().numpy()
        return pp, idx
    
    def __call__(self, x):
        '''
          input x is a 1D list that has the idx of the story
          which is normally from MyDataset
        '''
        x = torch.tensor(x.astype(np.int32))[None, :]
        N, L = x.shape
        if L < self.x_len:
            pad = torch.zeros(N, self.x_len-L, dtype=x.dtype) + self.pad_idx
            x = torch.cat([x, pad], dim=1)
        
        queue = []
        output = []
        y = torch.zeros(1, dtype=torch.long) + self.bos_idx
        heapq.heappush(queue, (0, y))
        
        while queue:
            p, y = heapq.heappop(queue)
            p = -p
            if y[-1] == self.eos_idx or y[-1] == self.pad_idx or len(y) > self.max_len:
                output.append((p, y))
                if len(output) == self.topk:
                    break
                continue
            
            for pp, idx in zip(*self._nexttoken(x, y[None, :])):
                y_new = torch.concat([y, torch.tensor([idx])])
                heapq.heappush(queue, (-(p+pp), y_new))
                if len(queue) > self.max_queue:
                    break
        
        return output

In [138]:
inference = BeamSearch(topk = 5, 
                       x_len = 1024,
                       max_len = 128, 
                       model = model, 
                       tokenizer = tokenizer,
                       max_queue = 20)

In [129]:
tokenizer.detokenize(x[0]), tokenizer.detokenize(y[0])

("a transportation security administration agent who was fired in nashville after becoming obsessed with a coworker - following her ,taking videos and photos and breaking into her house five times - was jailed this week as a result of the stalking case .daniel boykin ,33 ,plead guilty to charges of unlawful photography ,aggravated burglary ,wiretapping ,unlawful telephone recording and two computer crimes .in apologizing for his actions in court thursday ,boykin admitted to repeatedly filming the woman inside a bathroom at nashville international airport and was subsequently sentenced to six months prison .at the time of his arrest last year ,boykin was found with 92 videos of the victim  29 taken from the airport bathroom  and 1 ,527 photographs .investigators found additional images taken inside the woman 's home of air vents and shelves in the bathroom ,which they believe suggested he intended to plant hidden cameras at some point .obsessed :daniel boykin ,33 ,a former tsa employee 

In [130]:
tokenizer.detokenize(model(x[0:1,:], y[0:1,:-1]).argmax(dim=-1)[0])

"daniel boykin ,ed guilty to charges of unlawful photography ,aggravated burglary ,wiretapping ,unlawful telephone recording and boywas obsessed with a coemployee ,boywas a was sentenced king case dog being a of boykin phone s phone and boywas to videos of the victim '29 taken from the airport and nashville international airport .but was ,527 photographs .boytook into her home five times and stole photos of her clothing .<eos>authorities victim he he was 'sick shocked '<eos>to six months prison for a the times prison for <eos>the "

In [131]:
tokenizer.detokenize(y[0,:5])

'<bos>daniel boykin plead'

In [132]:
tokenizer.detokenize(model(x[0:1,:], y[0:1,:5]).argmax(dim=-1)[0])

'daniel boykin ,ed '

In [172]:
idx = 10
with torch.no_grad():
    print(f'story\n')
    print(tokenizer.detokenize(x[idx].numpy()))
    output = inference(x[idx].numpy())
    target = tokenizer.detokenize(y[idx,1:][w[idx]])
    print(f'target\n {target}')
    for pp, tokens in output:
        print(pp)
        print(tokenizer.detokenize(tokens))
    preds = tokenizer.detokenize(output[0][1][1:])
    print(f"pred\n {preds}")
    print(rouge_score(preds, target, rouge_keys=('rouge1','rouge2','rougeL')))

story

bolton fans were given their first taste of emile heskey in a wanderers shirt on monday evening as the veteran forward played an hour for their under 21 side .the 36-year-old managed 63 minutes in the 3-1 defeat by reading in the premier league cup before being replaced by jamie thomas .heskey has been training with the club since early december and is hopeful of securing a short-term deal with the championship side .emile heskey looks pained after missing a chance for bolton wanderers 'under 21 side on monday .heskey has been training with bolton since early december but this was his first game for the club .the former england international dribbles while being closed down by niall keown .the former england centre forward has been out of contract since leaving a-league side the newcastle jets at the end of their season and could not get the better of niall keown ,son of sportsmail columnist martin .however ,bolton manager neil lennon said on tuesday :'i was pleasantly surprised

In [153]:
from datetime import datetime

In [161]:
with torch.no_grad():
    rouge_1_sum, rouge_2_sum, rouge_L_sum, ntotal = 0.0, 0.0, 0.0, 0
    for x, y, w in validation_loader:
        for i in range(x.shape[0]):
            target = tokenizer.detokenize(y[i,1:][w[i]])
            output = inference(x[i].numpy())
            preds = tokenizer.detokenize(output[0][1][1:])
#             print(target)
#             print(preds)
#             print(rouge_score(preds, target, rouge_keys=('rouge1','rouge2')))
            score = rouge_score(preds, target, rouge_keys=('rouge1','rouge2', 'rougeL'))
            rouge_1_sum += score['rouge1_recall']
            rouge_2_sum += score['rouge2_recall']
            rouge_L_sum += score['rougeL_recall']
            ntotal += 1
        print(datetime.now())
        print(f"{rouge_1_sum / ntotal}, {rouge_2_sum / ntotal}, {rouge_L_sum / ntotal}")

2022-12-30 18:08:40.666045
0.3424490988254547, 0.13190190494060516, 0.2597731947898865
2022-12-30 18:11:42.521195
0.3308875560760498, 0.12419041991233826, 0.2535352110862732
2022-12-30 18:14:36.651442
0.3252846300601959, 0.1143350675702095, 0.24620306491851807
2022-12-30 18:17:25.300031
0.32439371943473816, 0.11838701367378235, 0.24760843813419342
2022-12-30 18:20:45.378227
0.31992146372795105, 0.11551003158092499, 0.2458784133195877
2022-12-30 18:23:32.824114
0.31899914145469666, 0.11630917340517044, 0.24449707567691803
2022-12-30 18:26:27.449448
0.31821122765541077, 0.11574387550354004, 0.244820699095726
2022-12-30 18:29:26.379701
0.321756511926651, 0.12073605507612228, 0.24934828281402588
2022-12-30 18:32:21.999356
0.32387012243270874, 0.12303820252418518, 0.2505589723587036
2022-12-30 18:35:09.259304
0.32046955823898315, 0.12024302780628204, 0.2484174221754074
2022-12-30 18:38:14.152169
0.31932327151298523, 0.11994259059429169, 0.2479528784751892
2022-12-30 18:41:23.760354
0.321294

[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [E

2022-12-30 21:23:38.864829
0.31495699286460876, 0.11929076910018921, 0.24537906050682068


[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [E

2022-12-30 21:26:18.764617
0.3148644268512726, 0.11929704248905182, 0.24538075923919678


...

2022-12-30 21:32:04.161272
0.31460100412368774, 0.11890220642089844, 0.24527256190776825


[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [E

2022-12-30 21:34:45.055340
0.3143572211265564, 0.11864088475704193, 0.24503663182258606


[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading punkt: <urlopen error [E

KeyboardInterrupt: 